In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from spectools import lpsd
from multiprocessing import Pool
pool = Pool()

In [ ]:
def fftnoise(f):
    f = np.array(f, dtype='complex')
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1:Np+1] *= phases
    f[-1:-1-Np:-1] = np.conj(f[1:Np+1])
    return np.fft.ifft(f).real
    
def band_limited_noise(min_freq, max_freq, samples=1024, samplerate=1):
    freqs = np.abs(np.fft.fftfreq(samples, 1/samplerate))
    f = np.zeros(samples)
    idx = np.where(np.logical_and(freqs>=min_freq, freqs<=max_freq))[0]
    f[idx] = 1
    return fftnoise(f)
    
N  = int(1e6)
fs = 2000

x1 = 200*band_limited_noise(200, 500, N, fs)
x2 = 20*band_limited_noise(100, 200, N, fs)

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Sample')
ax.set_ylabel('Signal')
ax.plot(x1, color='gray')
ax.plot(x2, color='k')
plt.show()

f1, ps1 = signal.welch(x1, fs=fs, nperseg=int(1e6/2), noverlap=int(1e6/4), scaling='spectrum', return_onesided=True)
f2, ps2 = signal.welch(x2, fs=fs, nperseg=int(1e6/2), noverlap=int(1e6/4), scaling='spectrum', return_onesided=True)
f3, csd12 = signal.csd(x1, x2, fs=fs, nperseg=int(1e6/2), noverlap=int(1e6/4), scaling='density', return_onesided=True)

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Power spectrum (${\rm units}^2/{\rm Hz}$)')
ax.loglog(f1, ps1, color='gray')
ax.loglog(f2, ps2, linestyle='--', color='gray')
plt.show()

In [ ]:
bmin = 0
order = -1

pso1 = lpsd.ltf(x1, fs=fs, bmin=bmin, order=order, pool=pool, win="Kaiser")
pso2 = lpsd.ltf(x2, fs=fs, bmin=bmin, order=order, pool=pool, win="Kaiser")
csd  = lpsd.ltf([x1,x2], fs=fs, bmin=bmin, order=order, pool=pool, win="Kaiser", verbose=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Power spectrums ${\rm units}^2/{\rm Hz}$')
ax.loglog(f1, ps1, label="Welch x1", color='gray', ls='-')
ax.loglog(f2, ps2, label="Welch x2", color='gray', ls='--')
ax.loglog(pso1.f, pso1.ps, label="LPSD x1", color='k', ls='-')
ax.loglog(pso2.f, pso2.ps, label="LPSD x2", color='k', ls='--')
ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Cross spectral density (units /Hz)')
ax.loglog(f3, np.abs(csd12), label='Welch', color='gray')
ax.loglog(csd.f, np.abs(csd.csd), label='LPSD', color='k')
ax.legend()
fig.tight_layout()
plt.show()